In [ ]:
import os
import random
import shutil
import time
from pathlib import Path
from typing import Dict, List, Tuple

import cv2
import keyboard
from PIL import Image, ImageTk
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import transforms
from torchvision.transforms import InterpolationMode

import tkinter as tk
from tkinter import Label, Button, Frame

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
data_path = Path("data/")
image_path = data_path / "rock_paper_scissors"

In [ ]:
classes = ["rock", "paper", "scissors"]
IMAGE_PER_CLASS = 300


def capture_dataset(cls, num_images):
    cap = cv2.VideoCapture(0)
    count = 0

    print(f"Press Enter to start capture for {cls}\nTilt and move your hand around")
    print("Press Esc to exit")
    keyboard.wait("enter")
    time.sleep(1)

    while count < num_images:
        ret, frame = cap.read()

        cv2.putText(frame, 
                    f"Class: {cls} | Image: {count+1}/{num_images}", 
                    (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, 
                    (0, 0, 0), 
                    2)
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            print("Exiting")
            break

        save_path = os.path.join(image_path, cls, f"{cls}_{count+1}.jpg")
        cv2.imwrite(save_path, frame)
        count += 1

    cap.release()
    cv2.destroyAllWindows()

if image_path.is_dir():
    print(f"Folder exists")
else:
    image_path.mkdir(parents=True, exist_ok=True)

    for cls in classes:
        os.makedirs(image_path / cls, exist_ok=True)
        capture_dataset(cls, IMAGE_PER_CLASS)

In [ ]:
train_dir = image_path/"train"
test_dir = image_path/"test"

if train_dir.is_dir() and test_dir.is_dir():
    print("Dataset is organised")
else:
    os.makedirs(train_dir, exist_ok = True)
    os.makedirs(test_dir, exist_ok = True)

    for cls in classes:
        images = [file for file in os.listdir(image_path/cls) if file.endswith(".jpg")]

        train_images, test_images = train_test_split(images, test_size=0.2, shuffle=True)

        os.makedirs(train_dir/cls, exist_ok=True)
        os.makedirs(test_dir/cls, exist_ok=True)

        for image in train_images:
            shutil.move(image_path/cls/image, train_dir/cls)

        for image in test_images:
            shutil.move(image_path/cls/image, test_dir/cls)

        shutil.rmtree(image_path/cls)

    print("Dataset organized")


In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
train_data = torchvision.datasets.ImageFolder(train_dir, auto_transforms)
test_data = torchvision.datasets.ImageFolder(test_dir, auto_transforms)

train_dataloader = DataLoader(
    dataset=train_data, shuffle=True, num_workers=os.cpu_count(), batch_size=32
)

test_dataloader = DataLoader(
    dataset=test_data, shuffle=False, num_workers=os.cpu_count(), batch_size=32
)

class_names = train_data.classes

train_dataloader, test_dataloader, class_names

In [ ]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
model.classifier.parameters

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=output_shape, bias=True),
).to(device)

In [ ]:
model.classifier.parameters

In [ ]:
summary(model,
    input_size=(32,3,224,224),
    verbose=0,
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
def train_step(
    model=nn.Module,
    dataloader=DataLoader,
    loss_fn=nn.Module,
    optimizer=torch.optim.Optimizer,
    device=torch.device,
) -> Tuple[float, float]:
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)

        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)

        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    train_loss = train_loss / len(dataloader)

    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device):

    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():

        for batch, (X, y) in enumerate(dataloader):

            X, y = X.to(device), y.to(device)

            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)

            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)

            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    test_loss = test_loss / len(dataloader)

    test_acc = test_acc / len(dataloader)

    return test_loss, test_acc

def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device,
) -> Dict[str, List]:

    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device
        )

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)

        results["train_acc"].append(train_acc)

        results["test_loss"].append(test_loss)

        results["test_acc"].append(test_acc)

    return results

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer

if os.path.exists("model.pth"):
    print("There is a trained model")
    model.load_state_dict(torch.load("model.pth"))
else:
    start_timer = timer()

    results = train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=5,
        device=device,
    )

    end_timer = timer()

    print(end_timer - start_timer)

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Model saved")

In [ ]:
def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(
            f"There are {len(dirnames)} directories and {len(filenames)}  images in '{dirpath}'."
        )

walk_through_dir(image_path)

In [ ]:

def predict(frame) -> str:
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(img)

    transform = transforms.Compose([
        transforms.Resize(256, interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  
                                std=[0.229, 0.224, 0.225])
    ])

    model.to(device)
    model.eval()  
    with torch.inference_mode():
        img_tensor = transform(pil_image).unsqueeze(0).to(device) 
        pred = model(img_tensor)
        pred_class =  torch.argmax(torch.softmax(pred, dim=1), dim=1)
    
    return class_names[pred_class]
        
def update_score(user_score, bot_score, prediction, bot_choice):

    result = ""

    if prediction == bot_choice:
        result = "Draw!"
    elif prediction == "scissors" and bot_choice == "rock":
        bot_score += 1
        result = "You Lose!"
    elif prediction == "rock" and bot_choice == "paper":
        bot_score += 1
        result = "You Lose!"
    elif prediction == "paper" and bot_choice == "scissors":
        bot_score += 1
        result = "You Lose!"

    elif prediction == "scissors" and bot_choice == "paper":
        user_score += 1
        result = "You Win!"
    elif prediction == "paper" and bot_choice == "rock":
        user_score += 1
        result = "You Win!"
    elif prediction == "rock" and bot_choice == "scissors":
        user_score += 1
        result = "You Win!"

    return user_score, bot_score, result

def random_bot_choice()-> str:
    bot_choice = random.choice(["rock", "scissors", "paper"])

    return bot_choice



In [ ]:
user_score, bot_score = 0, 0

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to access the webcam.")
    cap.release()
    exit()

def button_predict():
    global user_score, bot_score
    bot_choice = random_bot_choice()
    label_bot_choice.config(text=bot_choice)
    prediction = predict(cap.read()[1])
    label_predict.config(text=prediction)
    user_score, bot_score, result = update_score(user_score,bot_score, prediction, bot_choice)
    label_score.config(text= f"{user_score} : {bot_score}")
    label_result.config(text=result)
    
window = tk.Tk()
window.geometry("800x400")
window.title("Rock Scissors Paper")

label = Label(window)
label.pack(side="left", pady=50, padx=25)


def show_frames():
    ret, frame = cap.read()
    # print("Frame capture success:", ret, "Frame is None:", frame is None)  # Debug output
    if not ret or frame is None:
        print("Error: Unable to capture frame.")
        label.after(20, show_frames) 
        return
    try:
        frame = cv2.resize(frame, (400, 300))
    except Exception as e:
        print("Error during resizing:", e)
        label.after(20, show_frames)
        return
    try:
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
    except Exception as e:
        print("Error during image conversion:", e)
        label.after(20, show_frames)
        return
    label.imgtk = imgtk
    label.configure(image=imgtk)
    label.after(20, show_frames)

show_frames()


details = Frame(window)
details.pack(pady=50)


label_predict = Label(details, text = "Prediction", anchor='w', width=10)
label_predict.grid(row=0, column=0, sticky="w", padx=10, pady=10)

label_score = Label(details, text= f"{user_score} : {bot_score}", anchor='center', width=10)
label_score.grid(row=0, column=1, sticky="n", padx=10, pady=10)

label_bot_choice = Label(details, text="Bot", anchor='e', width=10)
label_bot_choice.grid(row=0, column=2, sticky="e", padx=10, pady=10)

label_result = Label(details, text="")
label_result.grid(row=1, column=1)

predict_button = Button(details, text="Predict", command=button_predict)
predict_button.grid(row=2, column=1, pady=20) 


window.grid_columnconfigure(0, weight=1) 
window.grid_columnconfigure(1, weight=1)  
window.grid_columnconfigure(2, weight=1)  
window.grid_rowconfigure(0, weight=1)  
window.grid_rowconfigure(1, weight=0)
window.grid_rowconfigure(1, weight=0)

window.mainloop()

cap.release()
cv2.destroyAllWindows()